In [1]:
file = open(r"original.cpp",'r')
#return_type = ("void","int","float","double","long long int","vector","string","stack")
updated_lines = []
change_words={}
for line in file:
    if(not line.startswith("//") and line!="\n"  and not line.startswith('#') and not line.startswith('using')):
        updated_lines.append(line.lower().strip())
    if(line.startswith("#define")):
        words = line.split(' ')
        #print(words)
        replace = words[1]
        string = ""
        for i in range(2,len(words)):
            string += words[i]
            string += " "
        string = string[:-2]
        #print(string)
        change_words[replace] = string

In [2]:
updated_lines

['double *a, *q, *r;',
 'double *c, *t;',
 'int n;',
 'void initvector(int *vector, int m);',
 'int main()',
 '{',
 'int i,j,k,m,thrid,num_threads;',
 'int qr_eval=1,eval=1; //debuging flags',
 'clock_t start, diff;',
 'double error,d,t_time,s_t_time;',
 'int *step;',
 '',
 '//cout<<"number of threads: ";',
 '//cin>>num_threads;',
 'num_threads=8;',
 'omp_set_num_threads(num_threads);',
 '',
 'a = (double *)malloc(n*n*sizeof(double));',
 't = (double *)malloc(n*n*sizeof(double));',
 'q = (double *)malloc(n*n*sizeof(double));',
 'r = (double *)malloc(n*n*sizeof(double));',
 'n=8;',
 'step=(int *)malloc(n*sizeof(int));',
 'initvector(step,n);',
 '//cout<<step[5];',
 'double r_sum = 0;',
 'cout<<"hello1"<<endl;',
 'srand(time(0));',
 '#pragma omp parallel private(i,j,k,thrid,r_sum)',
 '{',
 'thrid=omp_get_thread_num();',
 '//cout<<"hello"<<endl;',
 "//matrix a (initial), matrix 't' a temporary dublicate matrix",
 '//to keep the initial values of a',
 '//srand(t_time(0));',
 '#pragma omp f

In [3]:
#return_type = {"void ","int ","float ","double ","long long int ","vector ","string ","stack "}
code=""
for i in range(len(updated_lines)):
    code += updated_lines[i]

In [4]:
code

'double *a, *q, *r;double *c, *t;int n;void initvector(int *vector, int m);int main(){int i,j,k,m,thrid,num_threads;int qr_eval=1,eval=1; //debuging flagsclock_t start, diff;double error,d,t_time,s_t_time;int *step;//cout<<"number of threads: ";//cin>>num_threads;num_threads=8;omp_set_num_threads(num_threads);a = (double *)malloc(n*n*sizeof(double));t = (double *)malloc(n*n*sizeof(double));q = (double *)malloc(n*n*sizeof(double));r = (double *)malloc(n*n*sizeof(double));n=8;step=(int *)malloc(n*sizeof(int));initvector(step,n);//cout<<step[5];double r_sum = 0;cout<<"hello1"<<endl;srand(time(0));#pragma omp parallel private(i,j,k,thrid,r_sum){thrid=omp_get_thread_num();//cout<<"hello"<<endl;//matrix a (initial), matrix \'t\' a temporary dublicate matrix//to keep the initial values of a//srand(t_time(0));#pragma omp for //schedule(static,1)for (i = 0; i<n; i++){for(j=0;j<n;j++){a[i*n+j] = (rand()%10);t[i*n+j] = a[i*n+j];}step[i]=0;}cout<<"hello2"<<endl;//set lock = 0//unset lock = 1//para

In [5]:
change_words

{}

In [6]:
code = code.translate({ord("}"):"}; ",ord("("):" ( ",ord("{"):" { ;"})
for key,value in change_words.items():
    code=code.replace(key,value)

In [7]:
lines_code=code.split(';')

In [8]:
lines_code

['double *a, *q, *r',
 'double *c, *t',
 'int n',
 'void initvector ( int *vector, int m)',
 'int main ( ) { ',
 'int i,j,k,m,thrid,num_threads',
 'int qr_eval=1,eval=1',
 ' //debuging flagsclock_t start, diff',
 'double error,d,t_time,s_t_time',
 'int *step',
 '//cout<<"number of threads: "',
 '//cin>>num_threads',
 'num_threads=8',
 'omp_set_num_threads ( num_threads)',
 'a =  ( double *)malloc ( n*n*sizeof ( double))',
 't =  ( double *)malloc ( n*n*sizeof ( double))',
 'q =  ( double *)malloc ( n*n*sizeof ( double))',
 'r =  ( double *)malloc ( n*n*sizeof ( double))',
 'n=8',
 'step= ( int *)malloc ( n*sizeof ( int))',
 'initvector ( step,n)',
 '//cout<<step[5]',
 'double r_sum = 0',
 'cout<<"hello1"<<endl',
 'srand ( time ( 0))',
 '#pragma omp parallel private ( i,j,k,thrid,r_sum) { ',
 'thrid=omp_get_thread_num ( )',
 '//cout<<"hello"<<endl',
 "//matrix a  ( initial), matrix 't' a temporary dublicate matrix//to keep the initial values of a//srand ( t_time ( 0))",
 '#pragma omp fo

In [10]:
import re
from collections import Counter
pattern ="\((.*?)\)"
for i in range(len(lines_code)): 
    if ('{'in lines_code[i] and '(' in lines_code[i] and ')' in lines_code[i] and not('for ' in lines_code[i]) 
        and not('while ' in lines_code[i]) and not('main' in lines_code[i]) and not('if ' in lines_code[i])):
        
        substring = re.search(pattern, lines_code[i]).group(1)
        
        count_parameter=[]
        type_parameter=[]
        parameters = (substring.split(','))
        parameter_type = [i.rsplit(' ', 1)[0].strip() for i in parameters]
        count_parameter_all = dict(Counter(parameter_type))
        for key,value in count_parameter_all.items():
            count_parameter.append(value)
            type_parameter.append(key)
        
        
        words = lines_code[i].split(" ")
        if(words[0]=='static' or words[0]=='inline'):
            words[0]=''
        return_type=""
        for j in range(len(words)):
            if(words[j]=='('):
                for k in range(j-1):
                    return_type +=words[k]
                    return_type +=" "
                break

        function_signature=(return_type.strip(),count_parameter,type_parameter)
        print(function_signature)

('#pragma omp parallel', [1, 1, 1, 1, 1], ['', 'j', 'k', 'thrid', 'r_sum'])
('void xdoty', [3, 4], ['double', 'int'])
('void', [2], ['int'])
